In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geocoder
from geopy.geocoders import Nominatim


from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
%matplotlib inline

In [45]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# ypb = yelp business
ypb = pd.read_csv ('./data/yelp_business.csv')

In [3]:
ypb.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': ""u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ..."
2,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,45.511907,-122.613693,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Antiques, Fashion, Used, Vintage & Consignment...","{'Thursday': '11:0-18:0', 'Friday': '11:0-18:0..."
3,oaepsyvc0J17qwi8cfrOWg,Great Clips,2566 Enterprise Rd,Orange City,FL,32763,28.914482,-81.295979,3.0,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Beauty & Spas, Hair Salons",NaN
4,PE9uqAjdw0E4-8mjGl3wVA,Crossfit Terminus,1046 Memorial Dr SE,Atlanta,GA,30316,33.747027,-84.353424,4.0,14,1,"{'GoodForKids': 'False', 'BusinessParking': ""{...","Gyms, Active Life, Interval Training Gyms, Fit...","{'Monday': '16:0-19:0', 'Tuesday': '16:0-19:0'..."


In [4]:
ypb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160585 entries, 0 to 160584
Data columns (total 14 columns):
business_id     160585 non-null object
name            160585 non-null object
address         153859 non-null object
city            160585 non-null object
state           160585 non-null object
postal_code     160475 non-null object
latitude        160585 non-null float64
longitude       160585 non-null float64
stars           160585 non-null float64
review_count    160585 non-null int64
is_open         160585 non-null int64
attributes      145593 non-null object
categories      160470 non-null object
hours           133244 non-null object
dtypes: float64(3), int64(2), object(9)
memory usage: 17.2+ MB


In [5]:
ypb['address'] = ypb['address'].astype(str)

In [6]:
def title_maker(name):
    name = name.title().strip()
    return name
ypb['name'] = ypb['name'].map(title_maker)
ypb['address'] = ypb['address'].map(title_maker)
ypb['city'] = ypb['city'].map(title_maker)

In [8]:
ypb['addresses']=ypb['address'] + ', '+ ypb['city'] + ', ' + ypb['state']

In [9]:
ypb.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,addresses
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': ""u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...","921 Pearl St, Boulder, CO"
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants At Pdx,7000 Ne Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ...","7000 Ne Airport Way, Portland, OR"
2,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,45.511907,-122.613693,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Antiques, Fashion, Used, Vintage & Consignment...","{'Thursday': '11:0-18:0', 'Friday': '11:0-18:0...","4720 Hawthorne Ave, Portland, OR"
3,oaepsyvc0J17qwi8cfrOWg,Great Clips,2566 Enterprise Rd,Orange City,FL,32763,28.914482,-81.295979,3.0,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Beauty & Spas, Hair Salons",NaN,"2566 Enterprise Rd, Orange City, FL"
4,PE9uqAjdw0E4-8mjGl3wVA,Crossfit Terminus,1046 Memorial Dr Se,Atlanta,GA,30316,33.747027,-84.353424,4.0,14,1,"{'GoodForKids': 'False', 'BusinessParking': ""{...","Gyms, Active Life, Interval Training Gyms, Fit...","{'Monday': '16:0-19:0', 'Tuesday': '16:0-19:0'...","1046 Memorial Dr Se, Atlanta, GA"


In [61]:
smallb = pd.read_csv ('./data/smallb.csv')

/Users/songmei/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
df = ypb.merge(smallb, on='addresses')

In [62]:
df.head()

,business_id,name,address,city_x,state_x,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,addresses,Unnamed: 0,index,business_name,street_address,city_y,state_y,zip_code_first5,zip_code_last4,congressional_district,business_type,loan_amount,loan_net_proceeds_UNSURE,loan_size_rank_by_state,loan_amount_by_zip_percentile,loan_amount_by_type_percentile
0,HPA_qyMEddpAEtFof02ixg,Mr G'S Pizza & Subs,474 Lowell St,Peabody,MA,01960,42.541155,-70.973438,4.0,39,1,"{'RestaurantsGoodForGroups': 'True', 'HasTV': ...","Food, Pizza, Restaurants","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...","474 Lowell St, Peabody, MA",1686866,693021,M. Kathy Gabriel,474 Lowell St,Peabody,MA,1960,1385,6,PR,50000.0,6810.00,0.416911,0.783133,0.851531
1,HPA_qyMEddpAEtFof02ixg,Mr G'S Pizza & Subs,474 Lowell St,Peabody,MA,01960,42.541155,-70.973438,4.0,39,1,"{'RestaurantsGoodForGroups': 'True', 'HasTV': ...","Food, Pizza, Restaurants","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...","474 Lowell St, Peabody, MA",3564375,617222,Richard M. Danubio D.C.,474 Lowell St,Peabody,MA,1960,1385,6,PR,54100.0,7368.42,0.434038,0.843373,0.875510
2,u9cx2SH7OE_YBOYBcLoN_A,Ziggy & Sons' Donuts,474 Lowell St,Peabody,MA,01960,42.540980,-70.973846,4.5,32,1,"{'OutdoorSeating': 'False', 'BusinessParking':...","Coffee & Tea, Food, Donuts, Bakeries","{'Tuesday': '7:0-13:0', 'Wednesday': '7:0-13:0...","474 Lowell St, Peabody, MA",1686866,693021,M. Kathy Gabriel,474 Lowell St,Peabody,MA,1960,1385,6,PR,50000.0,6810.00,0.416911,0.783133,0.851531
3,u9cx2SH7OE_YBOYBcLoN_A,Ziggy & Sons' Donuts,474 Lowell St,Peabody,MA,01960,42.540980,-70.973846,4.5,32,1,"{'OutdoorSeating': 'False', 'BusinessParking':...","Coffee & Tea, Food, Donuts, Bakeries","{'Tuesday': '7:0-13:0', 'Wednesday': '7:0-13:0...","474 Lowell St, Peabody, MA",3564375,617222,Richard M. Danubio D.C.,474 Lowell St,Peabody,MA,1960,1385,6,PR,54100.0,7368.42,0.434038,0.843373,0.875510
4,l4b5g_MmMYGuiFSZy2N3zA,Levine'S Kosher Meat Market,474 Lowell St,Peabody,MA,01960,42.540960,-70.972802,3.5,11,1,"{'BusinessParking': ""{'garage': False, 'street...","Ethnic Food, Imported Food, Delis, Restaurants...",NaN,"474 Lowell St, Peabody, MA",1686866,693021,M. Kathy Gabriel,474 Lowell St,Peabody,MA,1960,1385,6,PR,50000.0,6810.00,0.416911,0.783133,0.851531


In [64]:
df.drop_duplicates(['business_id'], inplace=True)

In [65]:
df.drop(['city_y', 'state_y'], axis=1, inplace=True)

In [68]:
df.rename(columns={"city_x": "city", "state_x": "state"}, inplace=True)

In [69]:
df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,addresses,Unnamed: 0,index,business_name,street_address,zip_code_first5,zip_code_last4,congressional_district,business_type,loan_amount,loan_net_proceeds_UNSURE,loan_size_rank_by_state,loan_amount_by_zip_percentile,loan_amount_by_type_percentile
0,HPA_qyMEddpAEtFof02ixg,Mr G'S Pizza & Subs,474 Lowell St,Peabody,MA,01960,42.541155,-70.973438,4.0,39,1,"{'RestaurantsGoodForGroups': 'True', 'HasTV': ...","Food, Pizza, Restaurants","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...","474 Lowell St, Peabody, MA",1686866,693021,M. Kathy Gabriel,474 Lowell St,1960,1385,6,PR,50000.0,6810.0,0.416911,0.783133,0.851531
2,u9cx2SH7OE_YBOYBcLoN_A,Ziggy & Sons' Donuts,474 Lowell St,Peabody,MA,01960,42.540980,-70.973846,4.5,32,1,"{'OutdoorSeating': 'False', 'BusinessParking':...","Coffee & Tea, Food, Donuts, Bakeries","{'Tuesday': '7:0-13:0', 'Wednesday': '7:0-13:0...","474 Lowell St, Peabody, MA",1686866,693021,M. Kathy Gabriel,474 Lowell St,1960,1385,6,PR,50000.0,6810.0,0.416911,0.783133,0.851531
4,l4b5g_MmMYGuiFSZy2N3zA,Levine'S Kosher Meat Market,474 Lowell St,Peabody,MA,01960,42.540960,-70.972802,3.5,11,1,"{'BusinessParking': ""{'garage': False, 'street...","Ethnic Food, Imported Food, Delis, Restaurants...",NaN,"474 Lowell St, Peabody, MA",1686866,693021,M. Kathy Gabriel,474 Lowell St,1960,1385,6,PR,50000.0,6810.0,0.416911,0.783133,0.851531
6,4xBKnznDRCukFBQDJiNOhg,Family Fare Restaurant,474 Lowell St,Peabody,MA,01960,42.540915,-70.972857,3.0,37,1,"{'RestaurantsGoodForGroups': 'True', 'Restaura...","Coffee & Tea, Food, Breakfast & Brunch, Salad,...","{'Monday': '7:0-21:0', 'Tuesday': '7:0-21:0', ...","474 Lowell St, Peabody, MA",1686866,693021,M. Kathy Gabriel,474 Lowell St,1960,1385,6,PR,50000.0,6810.0,0.416911,0.783133,0.851531
8,wcjEH2OL19o4xDAvaOxsQQ,The Eye Place,474 Lowell St,Peabody,MA,01960,42.540941,-70.973114,4.0,8,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Health & Medical, Eyewear & Opticians, Shoppin...","{'Tuesday': '9:0-17:0', 'Wednesday': '9:0-17:0...","474 Lowell St, Peabody, MA",1686866,693021,M. Kathy Gabriel,474 Lowell St,1960,1385,6,PR,50000.0,6810.0,0.416911,0.783133,0.851531


In [70]:
df.drop(['loan_size_rank_by_state', 'loan_amount_by_zip_percentile','loan_amount_by_type_percentile'], axis=1, inplace=True)

In [72]:
df.drop(['attributes', 'hours','Unnamed: 0','index','zip_code_first5','zip_code_last4','is_open','review_count'], axis=1, inplace=True)

In [73]:
df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,categories,addresses,business_name,street_address,congressional_district,business_type,loan_amount,loan_net_proceeds_UNSURE
0,HPA_qyMEddpAEtFof02ixg,Mr G'S Pizza & Subs,474 Lowell St,Peabody,MA,01960,42.541155,-70.973438,4.0,"Food, Pizza, Restaurants","474 Lowell St, Peabody, MA",M. Kathy Gabriel,474 Lowell St,6,PR,50000.0,6810.0
2,u9cx2SH7OE_YBOYBcLoN_A,Ziggy & Sons' Donuts,474 Lowell St,Peabody,MA,01960,42.540980,-70.973846,4.5,"Coffee & Tea, Food, Donuts, Bakeries","474 Lowell St, Peabody, MA",M. Kathy Gabriel,474 Lowell St,6,PR,50000.0,6810.0
4,l4b5g_MmMYGuiFSZy2N3zA,Levine'S Kosher Meat Market,474 Lowell St,Peabody,MA,01960,42.540960,-70.972802,3.5,"Ethnic Food, Imported Food, Delis, Restaurants...","474 Lowell St, Peabody, MA",M. Kathy Gabriel,474 Lowell St,6,PR,50000.0,6810.0
6,4xBKnznDRCukFBQDJiNOhg,Family Fare Restaurant,474 Lowell St,Peabody,MA,01960,42.540915,-70.972857,3.0,"Coffee & Tea, Food, Breakfast & Brunch, Salad,...","474 Lowell St, Peabody, MA",M. Kathy Gabriel,474 Lowell St,6,PR,50000.0,6810.0
8,wcjEH2OL19o4xDAvaOxsQQ,The Eye Place,474 Lowell St,Peabody,MA,01960,42.540941,-70.973114,4.0,"Health & Medical, Eyewear & Opticians, Shoppin...","474 Lowell St, Peabody, MA",M. Kathy Gabriel,474 Lowell St,6,PR,50000.0,6810.0


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4054 entries, 0 to 5108
Data columns (total 17 columns):
business_id                 4054 non-null object
name                        4054 non-null object
address                     4054 non-null object
city                        4054 non-null object
state                       4054 non-null object
postal_code                 4054 non-null object
latitude                    4054 non-null float64
longitude                   4054 non-null float64
stars                       4054 non-null float64
categories                  4054 non-null object
addresses                   4054 non-null object
business_name               4054 non-null object
street_address              4054 non-null object
congressional_district      4054 non-null object
business_type               4054 non-null object
loan_amount                 4054 non-null float64
loan_net_proceeds_UNSURE    4054 non-null float64
dtypes: float64(5), object(12)
memory usage: 570.1+ KB


In [83]:
# Functions for Identifying How far a Customer's Patronage Supports Pandemic Business Recovery:
def zip_ranker(num):
    if num < 0.25:
        num = 'Village Friend: Making a Difference in this Zip'
        return num
    elif num >= 0.25 and num < 0.5:
        num = 'Village Benefactor: You\'re Making a Lasting Impact in this Zip'
        return num
    elif num >= 0.5 and num < 0.75:
        num = 'Village Patron: Your Impact is Going a Long Way in this Zip'
        return num
    elif num >= 0.75:
        num = 'Village Champ: Highest and Lasting Impact from Your Support in this Zip'
        return num
    else:
        num = 'Village Friend: Making a Difference in this Zip'
        return num
def type_ranker(num):
    if num < 0.25:
        num = 'Friend of the Business: Making a Difference for this Type'
        return num
    elif num >= 0.25 and num < 0.5:
        num = 'One of the Regulars: You\'re Making a Lasting Impact for this Type'
        return num
    elif num >= 0.5 and num < 0.75:
        num = 'Planting Roots: Your Impact is Going a Long Way for this Type'
        return num
    elif num >= 0.75:
        num = 'Business Partner: Highest and Lasting Impact from Your Support for this Type'
        return num
    else:
        num = 'Friend of the Business: Making a Difference for this Type'
        return num
def city_ranker(num):
    if num < 0.25:
        num = 'Friend of the City: Making a Difference in the Community'
        return num
    elif num >= 0.25 and num < 0.5:
        num = 'Locally Known: You\'re Making a Lasting Impact in the Community'
        return num
    elif num >= 0.5 and num < 0.75:
        num = 'Home Grown: Your Impact is Going a Long Way in the Community'
        return num
    elif num >= 0.75:
        num = 'Key to the City: Highest and Lasting Impact in the Community'
        return num
    else:
        num = 'Friend of the City: Making a Difference in the Community'
        return num
def full_loan_ranker(num):
    if num < 0.25:
        num = 'Village Friend: Making a Big Difference'
        return num
    elif num >= 0.25 and num < 0.5:
        num = 'Village Benefactor: Your Making a Lasting Impact'
        return num
    elif num >= 0.5 and num < 0.75:
        num = 'Village Patron: Your Impact is Going a Long Way in this Zip'
        return num
    elif num >= 0.75:
        num = 'Village Champ: Highest and Lasting Impact from Your Support'
        return num
    else:
        num = 'Village Friend: Making a Big Difference'
        return num
    
    
df['loan_size_rank_by_state'] = df['loan_amount'].rank(pct=True)
df['loan_amount_by_zip_percentile'] = df.groupby('postal_code')[['loan_amount']].rank(pct=True)
df['loan_amount_by_type_percentile'] = df.groupby('business_type')[['loan_amount']].rank(pct=True)
df['loan_amount_by_city_percentile'] = df.groupby('city')[['loan_amount']].rank(pct=True)
df['zipcode_urgency'] = df['loan_amount_by_zip_percentile'].apply(zip_ranker)
df['type_urgency'] = df['loan_amount_by_type_percentile'].apply(type_ranker)
df['city_urgency'] = df['loan_amount_by_city_percentile'].apply(city_ranker)
df['loan_size_urgency'] = df['loan_size_rank_by_state'].apply(full_loan_ranker)

In [91]:
df.head(100)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,categories,addresses,business_name,street_address,congressional_district,business_type,loan_amount,loan_net_proceeds_UNSURE,loan_size_rank_by_state,loan_amount_by_zip_percentile,loan_amount_by_type_percentile,loan_amount_by_city_percentile,zipcode_urgency,type_urgency,city_urgency,loan_size_urgency
0,HPA_qyMEddpAEtFof02ixg,Mr G'S Pizza & Subs,474 Lowell St,Peabody,MA,01960,42.541155,-70.973438,4.0,"Food, Pizza, Restaurants","474 Lowell St, Peabody, MA",M. Kathy Gabriel,474 Lowell St,6,PR,50000.0,6810.00,0.392699,0.837838,0.835948,0.844156,Village Champ: Highest and Lasting Impact from...,Business Partner: Highest and Lasting Impact f...,Key to the City: Highest and Lasting Impact in...,Village Benefactor: Your Making a Lasting Impact
2,u9cx2SH7OE_YBOYBcLoN_A,Ziggy & Sons' Donuts,474 Lowell St,Peabody,MA,01960,42.540980,-70.973846,4.5,"Coffee & Tea, Food, Donuts, Bakeries","474 Lowell St, Peabody, MA",M. Kathy Gabriel,474 Lowell St,6,PR,50000.0,6810.00,0.392699,0.837838,0.835948,0.844156,Village Champ: Highest and Lasting Impact from...,Business Partner: Highest and Lasting Impact f...,Key to the City: Highest and Lasting Impact in...,Village Benefactor: Your Making a Lasting Impact
4,l4b5g_MmMYGuiFSZy2N3zA,Levine'S Kosher Meat Market,474 Lowell St,Peabody,MA,01960,42.540960,-70.972802,3.5,"Ethnic Food, Imported Food, Delis, Restaurants...","474 Lowell St, Peabody, MA",M. Kathy Gabriel,474 Lowell St,6,PR,50000.0,6810.00,0.392699,0.837838,0.835948,0.844156,Village Champ: Highest and Lasting Impact from...,Business Partner: Highest and Lasting Impact f...,Key to the City: Highest and Lasting Impact in...,Village Benefactor: Your Making a Lasting Impact
6,4xBKnznDRCukFBQDJiNOhg,Family Fare Restaurant,474 Lowell St,Peabody,MA,01960,42.540915,-70.972857,3.0,"Coffee & Tea, Food, Breakfast & Brunch, Salad,...","474 Lowell St, Peabody, MA",M. Kathy Gabriel,474 Lowell St,6,PR,50000.0,6810.00,0.392699,0.837838,0.835948,0.844156,Village Champ: Highest and Lasting Impact from...,Business Partner: Highest and Lasting Impact f...,Key to the City: Highest and Lasting Impact in...,Village Benefactor: Your Making a Lasting Impact
8,wcjEH2OL19o4xDAvaOxsQQ,The Eye Place,474 Lowell St,Peabody,MA,01960,42.540941,-70.973114,4.0,"Health & Medical, Eyewear & Opticians, Shoppin...","474 Lowell St, Peabody, MA",M. Kathy Gabriel,474 Lowell St,6,PR,50000.0,6810.00,0.392699,0.837838,0.835948,0.844156,Village Champ: Highest and Lasting Impact from...,Business Partner: Highest and Lasting Impact f...,Key to the City: Highest and Lasting Impact in...,Village Benefactor: Your Making a Lasting Impact
10,buTr0hMiyDbjQvFJgm-Eew,Jp Licks,611 Assembly Row,Somerville,MA,02145,42.395406,-71.079491,3.5,"Custom Cakes, Desserts, Coffee & Tea, Ice Crea...","611 Assembly Row, Somerville, MA","J.P. Licks At Assembly Row, Inc",611 Assembly Row,7,R,25000.0,3405.00,0.264184,0.136364,0.171972,0.126667,Village Friend: Making a Difference in this Zip,Friend of the Business: Making a Difference fo...,Friend of the City: Making a Difference in the...,Village Benefactor: Your Making a Lasting Impact
11,QyZ0A3UUH_V4RoLAIrpwFw,J.P. Licks - Assembly Row,611 Assembly Row,Somerville,MA,02145,42.395540,-71.079465,3.0,"Food, Ice Cream & Frozen Yogurt, Coffee Roaste...","611 Assembly Row, Somerville, MA","J.P. Licks At Assembly Row, Inc",611 Assembly Row,7,R,25000.0,3405.00,0.264184,0.136364,0.171972,0.126667,Village Friend: Making a Difference in this Zip,Friend of the Business: Making a Difference fo...,Friend of the City: Making a Difference in the...,Village Benefactor: Your Making a Lasting Impact
12,XDv29FffNd2dWnDOtZP-wg,Sapporo Ramen,1815 Massachusetts Ave,Cambridge,MA,02140,42.387212,-71.118532,3.5,"Ramen, Restaurants, Japanese","1815 Massachusetts Ave, Cambridge, MA",Chochos,1815 Massachusetts Ave,5,PR,15000.0,2043.00,0.157745,0.326087,0.357516,0.182692,Village Benefactor: You're Making a Lasting Im...,One 

In [89]:
df.to_csv('./data/yelp_loan.csv')

In [90]:
df.to_json('./data/yelp_loan.json',orient='records')